## 내 집 마련을 위해, 서울 아파트 매물 데이터를 이용하여 구매 전략을 세우고자 한다.

real_estate.csv

|컬럼|정의|type|
|:---|:---|:---:|
|latitude|위도|float64|
|longitude|경도|float64|
|price_per_square_py|평당가|float64|
|py|평수|int64|
|apt_code|시공사 코드(길이5 알파벳 대문자)|object|
|dist_from_station|인근 지하철역과의 거리|float64|  

정답 및 해설 : https://tjd229.tistory.com/22

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('../content/tjd229/real_estate.csv')

### Q1. 시공사별 아파트 평수의 평균을 구하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 데이터의 첫 번째 행부터 열 번째 행까지의 시공사 코드(apt_code) 리스트를 저장한다. 첫 번째 행의 데이터는 위도, 경도, 평당가, 평수, 시공사 코드, 인근 지하철역과의 거리가 각각 37.125541, 126.913776, 2860.053787, 24, GEDAE, 1588.406226이다.  
단계 2 : 시공사 코드(apt_code)가 단계 1에서 구한 리스트에 포함되어 있는 데이터를 전부 제거한다.  
단계 3 : 시공사 코드(apt_code)별 평수 평균을 구하고 그 중 가장 큰 값을 구하시오  

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [4]:
df1 = df.copy()
df1.head(10)

,latitude,longitude,price_per_square_py,py,apt_code,dist_from_station
0,37.125541,126.913776,2860.053787,24,GEDAE,1588.406226
1,37.223169,127.026913,1715.073163,59,IHEAD,986.146800
2,37.239029,126.906303,1780.604630,57,FHIBJ,1492.514512
3,37.274433,126.737771,1508.814336,42,JBCIJ,1621.602866
4,37.082670,127.143634,2933.930488,32,EDJCI,945.034519
5,37.295509,126.825066,1698.052341,32,EHCAD,1565.151207
6,37.091887,127.124485,1511.888500,24,HEBIA,1212.276382
7,37.033688,126.996547,1566.233341,24,HEBIA,1550.428144
8,37.091141,126.978800,3503.553895,24,IIGFI,1656.802969
9,37.185720,126.876660,3149.065212,24,FHIBJ,1746.977072


### Q2. 아파트 매물을 분석하기 위해 군집화를 하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.


단계 1 : 독립 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2~9개 중 K-means Silhouette 를 통해 구하고, 이 중 첫번째로 높은 score를 최적의 K로 설정한다.  
            - 독립 변수(총 5개) : 시공사 코드(apt_code)를 제외한 모든 컬럼  

단계 2 : 단계 1에서 최적의 K로 도출한 각 군집의 비율 중, 가장 큰 값을 구하시오  

※ 정답은 올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.cluster import KMeans  
from sklearn.metrics import silhouette_score    
random_state = 229  
문제 지시 외 Default 값 사용  

### Q3. 아파트 가격을 예측하는 모델을 만드려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 평당가(price_per_square_py)와 평수(py)를 곱하여 실제 가격 변수를 생성한다.  
단계 2 : Train Set과 Test Set을 28/30:2/30 비율로 나눈다.  
단계 3 : Train Set으로 LinearRegression 모델을 학습하고, Test Set에 적용한다. 이 때, 절편을 포함한 경우와 포함하지 않은 경우 각각에 대하여 수행한다.  
            - 독립 변수(총 5개) : 시공사 코드(apt_code)와 총 가격을 제외한 모든 컬럼  
            - 종속 변수 : 실제 가격  

단계 3에서 학습한 각 모델에 대하여 예측 결과를 아래 정의된 Measure M으로 계산하였을 때 가장 큰 값은?  
$$ M = \biggl(\frac{1}{n} \sum_{i=1}^{n}{(y_i - \hat{y_i})^2}\biggr), \quad \hat{y_i}: 예측값, y_i: 실제값 $$

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split  
모든 random_state = 229  
문제 지시 외 Default 값 사용  